# 1. Project Packages

In [1]:
from operator import truediv
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report, cohen_kappa_score
from torchsummary import summary
from tqdm import tqdm, trange
import matplotlib.pyplot as plt
import torch.nn.functional as F
import torch.optim as optim
import scipy.io as sio
import torch.nn as nn
import torchvision
import numpy as np
import spectral
import torch
import math
import wget
import time
import os

name = "IP"
if (name == "IP" or name == "SA"):
    class_num = 16
elif (name == "PU"):
    class_num = 9

# 用于测试样本的比例
test_ratio = 0.85
# 每个像素周围提取 patch 的尺寸
patch_size = 23
# 使用 PCA 降维，得到主成分的数量
pca_components = 15

# 2. Proposed Model

In [2]:
class FE_HybridSN(nn.Module):
    def __init__(self):
        super(FE_HybridSN, self).__init__()
        self.conv3d_1 = nn.Sequential(
            nn.Conv3d(1, 8, kernel_size=(7, 3, 3), stride=1, padding=0),
            nn.BatchNorm3d(8),
            nn.ReLU(inplace=True),
        )
        self.conv3d_2 = nn.Sequential(
            nn.Conv3d(8, 16, kernel_size=(5, 3, 3), stride=1, padding=0),
            nn.BatchNorm3d(16),
            nn.ReLU(inplace=True),
        )
        self.conv3d_3 = nn.Sequential(
            nn.Conv3d(16, 32, kernel_size=(3, 3, 3), stride=1, padding=0),
            nn.BatchNorm3d(32), nn.ReLU(inplace=True))

        self.conv2d_1 = nn.Sequential(
            nn.Conv2d(96, 128, kernel_size=(3, 3), stride=1, padding=0),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
        )

        self.conv2d_2 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=(3, 3), stride=1, padding=0),
            nn.BatchNorm2d(256),
            nn.ReLU(inplace=True),
        )

        self.fc1 = nn.Linear(43264, 256)
        self.fc2 = nn.Linear(256, 128)
        self.fc3 = nn.Linear(128, class_num)
        self.dropout = nn.Dropout(p=0.4)

    def forward(self, x):
        out = self.conv3d_1(x)
        out = self.conv3d_2(out)
        out = self.conv3d_3(out)
        # out = self.conv3d_4(out)
        out = self.conv2d_1(out.reshape(out.shape[0], -1, 17, 17))
        out = self.conv2d_2(out.reshape(out.shape[0], -1, 15, 15))

        out = out.reshape(out.shape[0], -1)
        out = F.relu(self.dropout(self.fc1(out)))
        out = F.relu(self.dropout(self.fc2(out)))
        out = self.fc3(out)
        return out

# 3. Data Preprocessing Modules

## 3.1 PCA for Data Dimensional Reducing

In [3]:
# 对高光谱数据 X 应用 PCA 变换
def applyPCA(X, numComponents):
    newX = np.reshape(X, (-1, X.shape[2]))
    pca = PCA(n_components=numComponents, whiten=True)
    newX = pca.fit_transform(newX)
    newX = np.reshape(newX, (X.shape[0], X.shape[1], numComponents))
    return newX

## 3.2 Forming Patches of hyperspectral images

In [4]:
# 对单个像素周围提取 patch 时，边缘像素就无法取了，因此，给这部分像素进行 padding 操作
def padWithZeros(X, margin=2):
    newX = np.zeros(
        (X.shape[0] + 2 * margin, X.shape[1] + 2 * margin, X.shape[2]))
    x_offset = margin
    y_offset = margin
    newX[x_offset:X.shape[0] + x_offset, y_offset:X.shape[1] + y_offset, :] = X
    return newX

def createImageCubes(X, y, windowSize=5, removeZeroLabels=True):
    # 给 X 做 padding
    margin = int((windowSize - 1) / 2)
    zeroPaddedX = padWithZeros(X, margin=margin)
    # 获得 y 中的标记样本数---10249
    count = 0
    for r in range(0, y.shape[0]):
        for c in range(0, y.shape[1]):
            if y[r, c] != 0:
                count = count + 1

    # split patches
    patchesData = np.zeros([count, windowSize, windowSize, X.shape[2]])
    patchesLabels = np.zeros(count)

    count = 0
    for r in range(margin, zeroPaddedX.shape[0] - margin):
        for c in range(margin, zeroPaddedX.shape[1] - margin):
            if y[r - margin, c - margin] != 0:
                patch = zeroPaddedX[r - margin:r + margin + 1,
                                    c - margin:c + margin + 1]
                patchesData[count, :, :, :] = patch
                patchesLabels[count] = y[r - margin, c - margin]
                count = count + 1

    return patchesData, patchesLabels

## 3.3 Splitting Dataset into trainSet & testingSet 

In [5]:
def splitTrainTestSet(X, y, testRatio, randomState=345):
    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=testRatio, random_state=randomState, stratify=y)
    return X_train, X_test, y_train, y_test

## 3.4 Data Loading

In [6]:
def loadData(name):
    data_path = "D:\VsCode WorkSpace\Hybrid2D&3D\Data"
    if name == 'IP':
        data = sio.loadmat(
            os.path.join(data_path, 'Indian-Pines\Indian_pines_corrected.mat')
        )['indian_pines_corrected']
        labels = sio.loadmat(
            os.path.join(
                data_path,
                'Indian-Pines\Indian_pines_gt.mat'))['indian_pines_gt']
    elif name == 'SA':
        data = sio.loadmat(
            os.path.join(data_path,
                         'Salinas\Salinas_corrected.mat'))['salinas_corrected']
        labels = sio.loadmat(os.path.join(
            data_path, 'Salinas\Salinas_gt.mat'))['salinas_gt']
    elif name == 'PU':
        data = sio.loadmat(
            os.path.join(data_path, 'Pavia-University\PaviaU.mat'))['paviaU']
        labels = sio.loadmat(
            os.path.join(data_path,
                         'Pavia-University\PaviaU_gt.mat'))['paviaU_gt']
    else:
        print("NO DATASET")
        exit()
    return data, labels

## 3.5 Focal Loss

In [7]:
def focal_loss(input_values, gamma):
    """Computes the focal loss"""
    p = torch.exp(-input_values)
    #loss = (1 - p) ** gamma * input_values
    loss = (1 - p)**gamma * input_values * 10
    return loss.mean()


class FocalLoss(nn.Module):
    def __init__(self, weight=None, gamma=0.):
        super(FocalLoss, self).__init__()
        assert gamma >= 0
        self.gamma = gamma
        self.weight = weight

    def forward(self, input, target):
        return focal_loss(
            F.cross_entropy(input,
                            target,
                            reduction='none',
                            weight=self.weight), self.gamma)

# 4. Training Samples Initialization

In [8]:
X, y = loadData(name)

""" testing set """
class TestDS(torch.utils.data.Dataset):
    def __init__(self):
        self.len = Xtest.shape[0]
        self.x_data = torch.FloatTensor(Xtest)
        self.y_data = torch.LongTensor(ytest)

    def __getitem__(self, index):
        # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        # 返回文件数据的数目
        return self.len

""" training set """
class TrainDS(torch.utils.data.Dataset):
    def __init__(self):
        self.len = Xtrain.shape[0]
        self.x_data = torch.FloatTensor(Xtrain)
        self.y_data = torch.LongTensor(ytrain)

    def __getitem__(self, index):
        # 根据索引返回数据和对应的标签
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        # 返回文件数据的数目
        return self.len

print('Hyperspectral data shape: ', X.shape)  
print('Label shape: ', y.shape)  

print('\n... ... PCA tranformation ... ...')
X_pca = applyPCA(X, numComponents=pca_components)
print('Data shape after PCA: ', X_pca.shape)  

print('\n... ... create data cubes ... ...')
X_pca, y = createImageCubes(
    X_pca, y, windowSize=patch_size) 
print('Data cube X shape: ', X_pca.shape)  
print('Data cube y shape: ', y.shape)  

print('\n... ... create train & test data ... ...')
Xtrain, Xtest, ytrain, ytest = splitTrainTestSet(X_pca, y, test_ratio)
print('Xtrain shape: ', Xtrain.shape)  
print('Xtest  shape: ', Xtest.shape)  

# 改变 Xtrain, Ytrain 的形状，以符合 keras 的要求
Xtrain = Xtrain.reshape(-1, patch_size, patch_size, pca_components, 1)
Xtest = Xtest.reshape(-1, patch_size, patch_size, pca_components, 1)
print('before transpose: Xtrain shape: ', Xtrain.shape)  
print('before transpose: Xtest  shape: ', Xtest.shape)  

# 为了适应 pytorch 结构，数据要做 transpose
Xtrain = Xtrain.transpose(0, 4, 3, 1, 2)
Xtest = Xtest.transpose(0, 4, 3, 1, 2)
print('after transpose: Xtrain shape: ', Xtrain.shape)  
print('after transpose: Xtest  shape: ', Xtest.shape)  
"""在此之前都是对数据的预处理"""
###########数据加载loader############
# 创建 trainloader 和 testloader
trainset = TrainDS()
testset = TestDS()
train_loader = torch.utils.data.DataLoader(
    dataset=trainset,
    batch_size=53,  # 128,53
    shuffle=True,
    num_workers=0)
test_loader = torch.utils.data.DataLoader(
    dataset=testset,
    batch_size=53,  # 128,53
    shuffle=False,
    num_workers=0)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
one = torch.ones(53, dtype=torch.long).to(device)
two = torch.ones(17, dtype=torch.long).to(device)

# 网络放到GPU上,一些metric的设定
net = FE_HybridSN().to(device)
criterion = FocalLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0001)

Hyperspectral data shape:  (145, 145, 200)
Label shape:  (145, 145)

... ... PCA tranformation ... ...
Data shape after PCA:  (145, 145, 15)

... ... create data cubes ... ...
Data cube X shape:  (10249, 23, 23, 15)
Data cube y shape:  (10249,)

... ... create train & test data ... ...
Xtrain shape:  (1537, 23, 23, 15)
Xtest  shape:  (8712, 23, 23, 15)
before transpose: Xtrain shape:  (1537, 23, 23, 15, 1)
before transpose: Xtest  shape:  (8712, 23, 23, 15, 1)
after transpose: Xtrain shape:  (1537, 1, 15, 23, 23)
after transpose: Xtest  shape:  (8712, 1, 15, 23, 23)


# 5. Training

In [9]:
# 训练开始时间
start_time = time.time()

net.train()
total_loss = 0
# proc_bar = tqdm(range(150))
for epoch in range(150):
    # proc_bar.set_description(f"正处于第{epoch}回合：")
    for i, (inputs, labels) in enumerate(train_loader):
        inputs = inputs.to(device)
        labels = labels.to(device)
        try:
            labels = labels - one
        except:
            labels = labels - two
        #print(labels)
        # 优化器梯度归零
        optimizer.zero_grad()
        # 正向传播 +　反向传播 + 优化
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print('[Epoch: %d]   [loss avg: %.4f]   [current loss: %.4f]' %
          (epoch + 1, total_loss / (epoch + 1), loss.item()))

# proc_bar.close()

[Epoch: 1]   [loss avg: 639.8418]   [current loss: 15.3945]
[Epoch: 2]   [loss avg: 516.1397]   [current loss: 12.1494]
[Epoch: 3]   [loss avg: 436.1934]   [current loss: 7.8814]
[Epoch: 4]   [loss avg: 374.0865]   [current loss: 7.1725]
[Epoch: 5]   [loss avg: 325.4330]   [current loss: 3.7012]
[Epoch: 6]   [loss avg: 287.7841]   [current loss: 2.6539]
[Epoch: 7]   [loss avg: 257.6203]   [current loss: 2.3027]
[Epoch: 8]   [loss avg: 232.7676]   [current loss: 2.2801]
[Epoch: 9]   [loss avg: 211.4439]   [current loss: 1.2336]
[Epoch: 10]   [loss avg: 194.2815]   [current loss: 1.9638]
[Epoch: 11]   [loss avg: 179.7700]   [current loss: 0.9758]
[Epoch: 12]   [loss avg: 166.6831]   [current loss: 0.3556]
[Epoch: 13]   [loss avg: 155.4848]   [current loss: 0.5388]
[Epoch: 14]   [loss avg: 145.6167]   [current loss: 0.7174]
[Epoch: 15]   [loss avg: 137.0446]   [current loss: 1.1351]
[Epoch: 16]   [loss avg: 129.3220]   [current loss: 0.3409]
[Epoch: 17]   [loss avg: 122.5433]   [current l

## 5.1 Training time

In [10]:
end_time = time.time()
print(f"time cost:{(end_time-start_time)/60} min")
print('Finished Training')

time cost:6.1885708729426065 min
Finished Training


# 6. Saving & Outputing the model params of proposed model

In [15]:
# 指定模型保存的地址
# path = 'D:\VsCode WorkSpace\Assets\FEHN-FL\IP_Patch23\model' + '.pth'
# torch.save(net, path)
# # 输出模型参数
# summary(net, (1, 15, 23, 23))

# 7. Evaluating the proposed model

In [16]:
a = np.ones(Xtest.shape[0])  ##9225是Xtest.shape[0]
ytest = ytest - a

count = 0
# 模型测试
net.eval()
for inputs, _ in test_loader:
    inputs = inputs.to(device)
    outputs = net(inputs)
    outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
    if count == 0:
        y_pred_test = outputs
        count = 1
    else:
        y_pred_test = np.concatenate((y_pred_test, outputs))

# 8. Generating the report of evaluation

## 8.1 text report

In [17]:
# 生成分类报告
classification = classification_report(ytest, y_pred_test, digits=4)
print(classification)


def AA_andEachClassAccuracy(confusion_matrix):
    counter = confusion_matrix.shape[0]
    list_diag = np.diag(confusion_matrix)
    list_raw_sum = np.sum(confusion_matrix, axis=1)
    each_acc = np.nan_to_num(truediv(list_diag, list_raw_sum))
    average_acc = np.mean(each_acc)
    return each_acc, average_acc


def reports(test_loader, y_test, name):
    count = 0
    # 模型测试
    for inputs, _ in test_loader:
        inputs = inputs.to(device)
        outputs = net(inputs)
        outputs = np.argmax(outputs.detach().cpu().numpy(), axis=1)
        if count == 0:
            y_pred = outputs
            count = 1
        else:
            y_pred = np.concatenate((y_pred, outputs))

    if name == 'IP':
        target_names = [
            'Alfalfa', 'Corn-notill', 'Corn-mintill', 'Corn', 'Grass-pasture',
            'Grass-trees', 'Grass-pasture-mowed', 'Hay-windrowed', 'Oats',
            'Soybean-notill', 'Soybean-mintill', 'Soybean-clean', 'Wheat',
            'Woods', 'Buildings-Grass-Trees-Drives', 'Stone-Steel-Towers'
        ]
    elif name == 'SA':
        target_names = [
            'Brocoli_green_weeds_1', 'Brocoli_green_weeds_2', 'Fallow',
            'Fallow_rough_plow', 'Fallow_smooth', 'Stubble', 'Celery',
            'Grapes_untrained', 'Soil_vinyard_develop',
            'Corn_senesced_green_weeds', 'Lettuce_romaine_4wk',
            'Lettuce_romaine_5wk', 'Lettuce_romaine_6wk',
            'Lettuce_romaine_7wk', 'Vinyard_untrained',
            'Vinyard_vertical_trellis'
        ]
    elif name == 'PU':
        target_names = [
            'Asphalt', 'Meadows', 'Gravel', 'Trees', 'Painted metal sheets',
            'Bare Soil', 'Bitumen', 'Self-Blocking Bricks', 'Shadows'
        ]

    classification = classification_report(y_test,
                                           y_pred,
                                           target_names=target_names)
    oa = accuracy_score(y_test, y_pred)
    confusion = confusion_matrix(y_test, y_pred)
    each_acc, aa = AA_andEachClassAccuracy(confusion)
    kappa = cohen_kappa_score(y_test, y_pred)

    return classification, confusion, oa * 100, each_acc * 100, aa * 100, kappa * 100


# 将结果写在文件里
classification, confusion, oa, each_acc, aa, kappa = reports(
    test_loader, ytest, name)
classification = str(classification)
confusion = str(confusion)
file_name = "D:\VsCode WorkSpace\FEHN-FL\Assets\FEHN-CL\IP_Patch23\classification_report(0.85-loss).txt"

with open(file_name, 'w') as x_file:
    x_file.write('\n')
    x_file.write('{} Kappa accuracy (%)'.format(kappa))
    x_file.write('\n')
    x_file.write('{} Overall accuracy (%)'.format(oa))
    x_file.write('\n')
    x_file.write('{} Average accuracy (%)'.format(aa))
    x_file.write('\n')
    x_file.write('{} training time (min)'.format((end_time-start_time)/60))
    x_file.write('\n')
    x_file.write('\n')
    x_file.write('{}'.format(classification))
    x_file.write('\n')
    x_file.write('{}'.format(confusion))

              precision    recall  f1-score   support

         0.0     0.9750    1.0000    0.9873        39
         1.0     0.9923    0.9547    0.9731      1214
         2.0     0.9944    0.9986    0.9965       706
         3.0     0.9950    0.9900    0.9925       201
         4.0     0.9880    1.0000    0.9940       411
         5.0     0.9968    1.0000    0.9984       621
         6.0     1.0000    1.0000    1.0000        24
         7.0     1.0000    1.0000    1.0000       406
         8.0     0.9444    1.0000    0.9714        17
         9.0     1.0000    0.9818    0.9908       826
        10.0     0.9743    0.9981    0.9860      2087
        11.0     1.0000    0.9524    0.9756       504
        12.0     1.0000    1.0000    1.0000       174
        13.0     1.0000    1.0000    1.0000      1075
        14.0     0.9849    0.9939    0.9894       328
        15.0     0.7980    1.0000    0.8876        79

    accuracy                         0.9882      8712
   macro avg     0.9777   

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\VsCode WorkSpace\\FEHN-FL\\Assets\\FEHN-CL\\IP_Patch23\\classification_report(0.85-loss).txt'

## 8.2 images of prediction report

In [ ]:
# 显示结果
# load the original image

my_color4IP = np.array([[255, 255, 255], [223, 214, 212], [191, 173, 169],
                        [159, 132, 126], [128, 92, 84], [160, 133, 102],
                        [192, 174, 119], [224, 215, 136], [253, 255, 153],
                        [189, 242, 140], [125, 229, 127], [61, 216, 114],
                        [0, 202, 106], [0, 170, 202], [10, 134, 236],
                        [31, 91, 193], [51, 51, 153]])

# my_color4IP_predict = np.array([[51, 204, 255], [223, 214, 212], [191, 173, 169],
#                         [159, 132, 126], [128, 92, 84], [160, 133, 102],
#                         [192, 174, 119], [224, 215, 136], [253, 255, 153],
#                         [189, 242, 140], [125, 229, 127], [61, 216, 114],
#                         [0, 202, 106], [0, 170, 202], [10, 134, 236],
#                         [31, 91, 193], [51, 51, 153]])

my_color4IP_predict = my_color4IP


X, y = loadData(name)

height = y.shape[0]
width = y.shape[1]

X = applyPCA(X, numComponents=pca_components)
X = padWithZeros(X, patch_size // 2)

# 逐像素预测类别
outputs = np.zeros((height, width))
for i in range(height):
    for j in range(width):
        if int(y[i, j]) == 0:
            continue
        else:
            image_patch = X[i:i + patch_size, j:j + patch_size, :]
            image_patch = image_patch.reshape(1, image_patch.shape[0],
                                              image_patch.shape[1],
                                              image_patch.shape[2], 1)
            X_test_image = torch.FloatTensor(
                image_patch.transpose(0, 4, 3, 1, 2)).to(device)
            prediction = net(X_test_image)
            prediction = np.argmax(prediction.detach().cpu().numpy(), axis=1)
            outputs[i][j] = prediction + 1
    if i % 20 == 0:
        print('... ... row ', i, ' handling ... ...')

# oringal_image = spectral.imshow(classes=y, figsize=(5, 5))
predict_image = spectral.imshow(classes=outputs.astype(int), figsize=(5, 5))
# spectral.save_rgb("D:\VsCode WorkSpace\FEHN-FL\Pics\IP-15%-FEHN\(PU-3-3-0.85)FE-NET-原始.eps",
#                   y.astype(int),
#                   colors=my_color4IP)
spectral.save_rgb(r"D:\VsCode WorkSpace\FEHN-FL\Pics\IP-15%-FEHN\fe.eps",
                  outputs.astype(int),
                  colors=my_color4IP_predict)